# <center>Rocket Lab Dados - 2025.2</center>
# <center> Introdução à Pyspark</center>
___
Todo o conteúdo que você terá acesso ao longo desse período é confidencial, não sendo possível compartilhar ou comercializar os links ou os materiais recebidos que sejam de propriedade do Programa Rocket Lab da V(dev)

Dessa forma, ao participar do curso você está aceitando os termos de confidencialidade e não-comercialização dos conteúdos que serão recebidos.
___

# <center> Objetivos de aprendizado </center>
- Familiarizar-se com as funcionalidades básicas do PySpark
- Ser capaz de carregar dados em um DataFrame
- Ser capaz de realizar manipulações básicas de dados
___


### 1. Juntando DataFrames

É muito comum ter a necessidade de juntar *DataFrames* diferentes. Se você já utilizou SQL ou qualquer outro banco de dados relacional, deve conhecer isso como *join*. O Pandas também tem a mesma função utilizando o método *.merge()*. Antes do exemplo, vamos aprender/relembrar os tipos de *joins* mais comuns:<br>
![Joining Methods](https://i.imgur.com/HaSBT91.jpg) <br>
Agora, vamos carregar um DataFrame mais simples para testar os tipos de *merge*.

Para os exemplos abaixo iremos utilizar o Datafram: **metal_bands**, contendo as informações sobre bandas de metal do mundo todo, suas origens e estilos musicais.

Principais colunas:
- Band — nome da banda
- Origin — país de origem
- Fans — número aproximado de fãs
- Formed — ano de formação
- Split — ano de separação ('-', se ainda ativa)
- Style — subgênero do metal (ex: Heavy Metal, Black Metal, Thrash Metal)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#Criar sessão spark
spark = SparkSession.builder.appName("AtividadePraticaSpark").getOrCreate()

# Execute esta célula para carregar o dataframe metal_bands com dados de bandas de metal
metal_bands = spark.table("workspace.bronze.metal_bands")

metal_bands.printSchema()
display(metal_bands.limit(5))

root
 |-- id: long (nullable = true)
 |-- band_name: string (nullable = true)
 |-- fans: long (nullable = true)
 |-- formed: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- split: string (nullable = true)
 |-- style: string (nullable = true)



id,band_name,fans,formed,origin,split,style
0,Iron Maiden,4195,1975,United Kingdom,-,"New wave of british heavy,Heavy"
1,Opeth,4147,1990,Sweden,1990,"Extreme progressive,Progressive rock,Progressive"
2,Metallica,3712,1981,USA,-,"Heavy,Bay area thrash"
3,Megadeth,3105,1983,USA,1983,"Thrash,Heavy,Hard rock"
4,Amon Amarth,3054,1988,Sweden,-,Melodic death


Vamos separar alguns dataframes a partir de *metal_bands* para testar os merges. Observe a célula abaixo.

In [0]:
# ano de formação e país das bandas
bands_origin = metal_bands.select('id','band_name','formed','origin')

# estilo das bandas
bands_style = metal_bands.select('id','band_name','style') # estilo das bandas

# bandas que se separaram
bands_split = (metal_bands
               .select('id','band_name','split')
               .where(F.column('split') != "-")
               )

# bandas com mais de 4000 fans
bands_4000_fans = (metal_bands
                   .select('id','band_name','fans')
                   .where(F.column('fans') > 4000)
                   )

# bandas formadas nos EUA
bands_USA = (metal_bands
             .select('id','band_name','formed','origin')
             .where(F.column('origin') == "USA")
             )

# bandas formadas na Suécia
bands_Sweden = (metal_bands
                .select('id','band_name','formed','origin')
                .where(F.column('origin') == 'Sweden')
                )

Vamos criar um DataFrame a partir de ```bands_origin``` e ```bands_split```, utilizando *merge*.

In [0]:
origin_split = (bands_origin # o DataFrame da esquerda
                .join(bands_split, # o DataFrame da direita
                      on=['id', 'band_name'], # baseado em quais valores em comum (chave)
                      how='inner' # o tipo de join que queremos fazer
                      )
                )
display(origin_split.limit(5))

id,band_name,formed,origin,split
1,Opeth,1990,Sweden,1990
3,Megadeth,1983,USA,1983
5,Slayer,1981,USA,1981
6,Death,1983,USA,2001
7,Dream Theater,1985,USA,1985


Ótimo! Conseguimos fazer o *Join* de dois *DataFrames*. Observe que utilizamos o argumento ```how='inner'```. Lembre-se que *inner*, *left*, *right* e *outer* terão resultados diferentes, observe os merges abaixo e a explicação ao final.

In [0]:
left_origin_split = (bands_origin
                     .join(bands_split,
                           on= ['id', 'band_name'],
                           how="left"
                           )
                     )
display(left_origin_split.limit(5))

id,band_name,formed,origin,split
0,Iron Maiden,1975,United Kingdom,null
1,Opeth,1990,Sweden,1990
2,Metallica,1981,USA,null
3,Megadeth,1983,USA,1983
4,Amon Amarth,1988,Sweden,null


In [0]:
right_origin_split = (bands_origin
                     .join(bands_split,
                           on= ['id', 'band_name'],
                           how="right"
                           )
                     )
display(right_origin_split.limit(5))

id,band_name,formed,origin,split
1,Opeth,1990,Sweden,1990
3,Megadeth,1983,USA,1983
5,Slayer,1981,USA,1981
6,Death,1983,USA,2001
7,Dream Theater,1985,USA,1985


In [0]:
print('Numero de linhas do DataFrame bands_4000_fans:', bands_4000_fans.count())
print('Numero de linhas do DataFrame bands_USA:', bands_USA.count())
print('----------------------------------------------')

outer_origin_split = (bands_4000_fans
                     .join(bands_USA,
                           on= ['id', 'band_name'],
                           how="outer"
                           )
                     )

print('Numero de linhas do DataFrame após Outer entre bands_4000_fans & bands_USA:', outer_origin_split.count())
display(outer_origin_split.limit(5))

Numero de linhas do DataFrame bands_4000_fans: 4
Numero de linhas do DataFrame bands_USA: 1139
----------------------------------------------
Numero de linhas do DataFrame após Outer entre bands_4000_fans & bands_USA: 1143


id,band_name,fans,formed,origin
763,The Absence,null,2002,USA
3417,Hail!Hornet,null,2006,USA
103,Nevermore,null,1991,USA
3933,Kowloon Walled City,null,2007,USA
1812,Black Fast,null,2010,USA


Como podemos ver, os resultados são de fato bem diferentes.

O *inner* mantém apenas os dados das bandas encontradas nos dois dataframes (onde há correspondência de *id*), dessa forma, a posição do dataframe não faz diferença.

No *left*, mantemos os dados do dataframe à esquerda, e trazemos os dados do dataframe à direita no qual encontrou-se a chave (neste exemplo, o *id* da banda).

Por outro lado, no *right* ocorre o contrário, mantemos os dados do dataframe à direita e, quando há correspondência da chave, trazemos os dados do dataframe à esquerda. Note que o número de entradas (*entries*) é diferente do caso com o *left*. Isso ocorre porque no *left* mantemos os dados de formação das bandas (ou seja, o dataframe contém todas as bandas do .csv), enquanto no *right*, mantemos apenas os dados de bandas que se separaram (e existem muitas bandas que ainda continuam juntas).

Por fim, no *outer* utilizamos dois dataframes diferentes dos anteriores para facilitar o entendimento. Observe pelos prints que existem apenas 4 bandas com mais de 4000 fans e 1139 bandas formadas nos EUA. Quando fazemos o *join* com *outer*, observe que o total de linhas passa a ser 1143. O que acontece é que esse tipo de join mantém os dados de ambos os dataframes, independente se houve correspondência de chave ou não.

Podemos também querer apenas concatenar dois *DataDrames*, isto é, juntá-los colocando um abaixo do outro. Para isso, utilizamos o método *.union()*:

In [0]:
# concatenando bandas formadas nos EUA e bandas formadas na Suécia
USA_Sweden = bands_USA.union(bands_Sweden)

print('Numero de linhas do DataFrame bands_USA:', bands_USA.count())
print('Numero de linhas do DataFrame bands_Sweden:', bands_Sweden.count())
print('Numero de linhas do DataFrame após union entre bands_USA & bands_Sweden:', USA_Sweden.count())
display(USA_Sweden.limit(5))

Numero de linhas do DataFrame bands_USA: 1139
Numero de linhas do DataFrame bands_Sweden: 476
Numero de linhas do DataFrame após union entre bands_USA & bands_Sweden: 1615


id,band_name,formed,origin
2,Metallica,1981,USA
3,Megadeth,1983,USA
5,Slayer,1981,USA
6,Death,1983,USA
7,Dream Theater,1985,USA


## Exercício 1
O Ultimate Team (FUT) é um modo do jogo FIFA no qual o jogador monta seu próprio time adquirindo atletas virtuais.
Cada atleta possui atributos que influenciam seu desempenho em campo — como drible, chute, passe, defesa, velocidade e físico.

Principais colunas:
- `player_id` — identificador único do jogador
- `player_name` — nome do atleta
- `nationality` — país de origem
- `club` — clube atual
- `overall` — nota geral do jogador
- `potential` — potencial máximo de evolução
- `value_eur, wage_eur` — valor de mercado e salário
- `age, height_cm, weight_kg` — características físicas
- `pace, shooting, passing, dribbling, defending, physic` — atributos técnicos

_**Preencha os espacos ____ para carregar os dados e realizar as consultas propostas.**_

### Exercício 1.1 Faça a leitura do arquivo fut_players (fut_player_data.csv) e retorne as 5 primeiras linhas

In [0]:
# Faça a leitura do arquivo fut_players 
fut_players = metal_bands = spark.table("workspace.bronze.fut_players_data")

# Retorne as 5 primeiras linhas do DF
display(fut_players.limit(5))

player_id,player_name,player_extended_name,quality,revision,origin,overall,club,league,nationality,position,age,date_of_birth,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,gk_diving,gk_reflexes,gk_handling,gk_speed,gk_kicking,gk_positoning,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,cb,rb,lb,rwb,lwb,cdm,cm,rm,lm,cam,cf,rf,lf,rw,lw,st,traits,specialities,base_id
1,Pelé,Arantes Nascimento Edson,Gold - Rare,Icon,Prime,98,Icons,Icons,Brazil,CAM,78,1940-10-23,173,70,0,2018-09-19,95.0,95,95,96.0,94,93,98,97,96,98,96.0,97,98,94,94,95,93,93.0,97,90,89,96,88,89,60.0,67,94,55,53,49,76.0,88,86,76,59,null,null,null,null,null,null,Right,High,Med,4,5,70.0,77.0,77.0,79.0,79.0,77.0,91.0,95.0,95.0,96.0,96.0,96.0,96.0,96.0,96.0,95.0,Finesse Shot,"Speedster, Aerial Threat, Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward, Poacher",237067
2,Maradona,Diego Maradona,Gold - Rare,Icon,Prime,97,Icons,Icons,Argentina,CAM,58,1960-10-30,165,70,0,2018-09-19,92.0,94,90,97.0,91,98,94,97,98,95,93.0,92,97,85,94,88,94,92.0,95,88,96,93,89,96,40.0,44,67,27,42,37,76.0,82,78,75,76,null,null,null,null,null,null,Left,High,Med,3,5,57.0,66.0,66.0,71.0,71.0,69.0,88.0,93.0,93.0,95.0,94.0,94.0,94.0,94.0,94.0,90.0,"Avoids Using Weaker Foot, Finesse Shot, Flair, Takes Finesse Free Kicks","Speedster, Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward",190042
3,Ronaldo,Nazário de Lima Ronaldo Luís,Gold - Rare,Icon,Prime,96,Icons,Icons,Brazil,ST,42,1976-09-22,183,78,0,2018-09-19,97.0,97,97,95.0,91,85,95,96,97,89,95.0,95,98,93,90,96,89,81.0,81,75,90,86,73,87,45.0,41,84,39,44,38,76.0,82,81,85,47,null,null,null,null,null,null,Right,Med,Med,5,5,60.0,67.0,67.0,70.0,70.0,66.0,83.0,90.0,90.0,91.0,94.0,94.0,94.0,92.0,92.0,94.0,"Tries To Beat Defensive Line, Finesse Shot","Speedster, Dribbler, Distance Shooter, FK Specialist, Acrobat, Clinical Finisher, Complete Forward, Poacher",37576
4,Pelé,Arantes Nascimento Edson,Gold - Rare,Icon,Medium,95,Icons,Icons,Brazil,CF,78,1940-10-23,173,69,0,2018-09-19,96.0,95,96,95.0,96,95,95,95,94,95,93.0,94,95,90,91,93,91,90.0,93,88,88,91,87,85,56.0,64,89,51,49,45,75.0,89,87,74,57,null,null,null,null,null,null,Right,High,Med,4,5,66.0,74.0,74.0,77.0,77.0,74.0,88.0,93.0,93.0,93.0,94.0,94.0,94.0,94.0,94.0,92.0,Finesse Shot,"Speedster, Dribbler, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Complete Forward, Poacher",237068
5,Maradona,Diego Maradona,Gold - Rare,Icon,Medium,95,Icons,Icons,Argentina,CAM,58,1960-10-30,165,72,0,2018-09-19,88.0,91,85,95.0,89,98,93,95,96,95,91.0,92,94,83,91,86,92,90.0,95,87,93,90,87,94,42.0,46,67,30,44,39,75.0,80,75,74,76,null,null,null,null,null,null,Left,High,Med,3,5,58.0,66.0,66.0,70.0,70.0,69.0,86.0,91.0,91.0,93.0,92.0,92.0,92.0,92.0,92.0,88.0,"Avoids Using Weaker Foot, Finesse Shot, Flair, Takes Finesse Free Kicks","Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder",237074


### Exercício 1.2 - Retorna a nacionalidade dos jogadores "The Bests"

São considerados jogadores The Bests os que possuem os atributos de drible (_dribbling_) e chute (_shooting_) superior a 90. 
Após a geração do DF _The_Best_ realize o join com o df _nationalities_ para obter a nacionalidade dos jogadores.

A sua tabela final deve conter as seguintes informações:
- `player_id`
- `player_name`
- `nationality`
- `position`
- `dribbling`
- `shooting`
- `overall`

In [0]:
# Aplique os filtros para retornar os jogadores the bests
the_best = (
    fut_players.where(
        (F.column('dribbling') > 90) &
        (F.column('shooting') > 90)
    )
)

# nationalities é um DataDrame da nacionalidade dos jogadores
nationalities = (fut_players.select('player_id', 'player_name', 'nationality'))

# faça um join dos dois DataDrames, mantendo todos os jogadores de the_best e obtendo suas nacionalidades (dica: a chave é o id)
the_best_nationality = (
    the_best.join(
        nationalities,
        on='player_id',
        how='left'
    )
)

the_best_nationality.display()

player_id,player_name,player_extended_name,quality,revision,origin,overall,club,league,nationality,position,age,date_of_birth,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,gk_diving,gk_reflexes,gk_handling,gk_speed,gk_kicking,gk_positoning,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,cb,rb,lb,rwb,lwb,cdm,cm,rm,lm,cam,cf,rf,lf,rw,lw,st,traits,specialities,base_id,player_name,nationality
1,Pelé,Arantes Nascimento Edson,Gold - Rare,Icon,Prime,98,Icons,Icons,Brazil,CAM,78,1940-10-23,173,70,0,2018-09-19,95.0,95,95,96.0,94,93,98,97,96,98,96.0,97,98,94,94,95,93,93.0,97,90,89,96,88,89,60.0,67,94,55,53,49,76.0,88,86,76,59,null,null,null,null,null,null,Right,High,Med,4,5,70.0,77.0,77.0,79.0,79.0,77.0,91.0,95.0,95.0,96.0,96.0,96.0,96.0,96.0,96.0,95.0,Finesse Shot,"Speedster, Aerial Threat, Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward, Poacher",237067,Pelé,Brazil
2,Maradona,Diego Maradona,Gold - Rare,Icon,Prime,97,Icons,Icons,Argentina,CAM,58,1960-10-30,165,70,0,2018-09-19,92.0,94,90,97.0,91,98,94,97,98,95,93.0,92,97,85,94,88,94,92.0,95,88,96,93,89,96,40.0,44,67,27,42,37,76.0,82,78,75,76,null,null,null,null,null,null,Left,High,Med,3,5,57.0,66.0,66.0,71.0,71.0,69.0,88.0,93.0,93.0,95.0,94.0,94.0,94.0,94.0,94.0,90.0,"Avoids Using Weaker Foot, Finesse Shot, Flair, Takes Finesse Free Kicks","Speedster, Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward",190042,Maradona,Argentina
3,Ronaldo,Nazário de Lima Ronaldo Luís,Gold - Rare,Icon,Prime,96,Icons,Icons,Brazil,ST,42,1976-09-22,183,78,0,2018-09-19,97.0,97,97,95.0,91,85,95,96,97,89,95.0,95,98,93,90,96,89,81.0,81,75,90,86,73,87,45.0,41,84,39,44,38,76.0,82,81,85,47,null,null,null,null,null,null,Right,Med,Med,5,5,60.0,67.0,67.0,70.0,70.0,66.0,83.0,90.0,90.0,91.0,94.0,94.0,94.0,92.0,92.0,94.0,"Tries To Beat Defensive Line, Finesse Shot","Speedster, Dribbler, Distance Shooter, FK Specialist, Acrobat, Clinical Finisher, Complete Forward, Poacher",37576,Ronaldo,Brazil
4,Pelé,Arantes Nascimento Edson,Gold - Rare,Icon,Medium,95,Icons,Icons,Brazil,CF,78,1940-10-23,173,69,0,2018-09-19,96.0,95,96,95.0,96,95,95,95,94,95,93.0,94,95,90,91,93,91,90.0,93,88,88,91,87,85,56.0,64,89,51,49,45,75.0,89,87,74,57,null,null,null,null,null,null,Right,High,Med,4,5,66.0,74.0,74.0,77.0,77.0,74.0,88.0,93.0,93.0,93.0,94.0,94.0,94.0,94.0,94.0,92.0,Finesse Shot,"Speedster, Dribbler, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Complete Forward, Poacher",237068,Pelé,Brazil
5,Maradona,Diego Maradona,Gold - Rare,Icon,Medium,95,Icons,Icons,Argentina,CAM,58,1960-10-30,165,72,0,2018-09-19,88.0,91,85,95.0,89,98,93,95,96,95,91.0,92,94,83,91,86,92,90.0,95,87,93,90,87,94,42.0,46,67,30,44,39,75.0,80,75,74,76,null,null,null,null,null,null,Left,High,Med,3,5,58.0,66.0,66.0,70.0,70.0,69.0,86.0,91.0,91.0,93.0,92.0,92.0,92.0,92.0,92.0,88.0,"Avoids Using Weaker Foot, Finesse Shot, Flair, Takes Finesse Free Kicks","Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder",237074,Maradona,Argentina
7,Ronaldo,Nazário de Lima Ronaldo Luís,Gold - Rare,Icon,Medium,94,Icons,Icons,Brazil,ST,42,1976-09-22,183,82,0,2018-09-19,93.0,94,93,93.0,87,81,94,94,95,91,93.0,96,96,91,89,90,90,80.0,84,72,90,84,72,84,46.0,43,81,39,46,38,80.0,81,79,89,58,null,null,null,null,null,null,Right,Med,Low,5,5,61.0,66.0,66.0,69.0,69.0,66.0,82.0,89.0,89.0,90.0,92.0,92.0,92.0,91.0,91.0,92.0,Fin

## 2. Alterando o dataframe


Agora iremos utilizar o DataFrame _**pokemon_data**_. Essa base reúne informações sobre os Pokémons das diversas gerações da franquia, contendo atributos, classificações e estatísticas de batalha.

Principais Colunas:
- Name — nome do Pokémon 
- Type 1, Type 2 — tipos primário e secundário (ex: Fire, Water, Grass) 
- HP, Attack, Defense, Sp. Atk, Sp. Def, Speed — atributos de combate 
- Generation — geração à qual pertence
- Legendary - Se e ou não um Pokémon lendário

In [0]:
pkmn = spark.table("workspace.bronze.pokemon_data")

display(pkmn.limit(5))

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,false
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,false
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,false
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,false
4,Charmander,Fire,null,309,39,52,43,60,50,65,1,false


Até o momento apenas utilizamos os dados da forma que nos foram fornecidos, mas e se precisássemos criar alguma coluna que fosse a combinação das demais? Por exemplo, caso eu deseje criar uma coluna que corresponde à soma do ataque e velocidade dos Pokémons? Observe abaixo:

In [0]:
# Criando a coluna desejada
pkmn = pkmn.withColumn("Sum_Attack_Speed", F.col("Attack") + F.col("Speed"))
display(pkmn.limit(5))

#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Sum_Attack_Speed
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,false,94
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,false,122
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,false,162
3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,false,180
4,Charmander,Fire,null,309,39,52,43,60,50,65,1,false,117


Observe como foi fácil! Apenas utilizamos o operador de soma com as duas colunas necessárias. Você pode fazer isso com outras operações também, basta utilizar ```-```, ```/``` ou ```*```. Além disso, você pode combinar quantas colunas quiser!

Mas e se precisarmos alterar apenas algumas linhas do nosso DataFrame?

Por exemplo, suponha que você percebeu que seus dados estão errados, e todos os Pokémons com velocidade acima de 100 deveriam estar marcados como Type_1 = 'Fire', podemos seguir o procedimento abaixo:

In [0]:
# Observe os valores unicos da coluna Type_1 para os Pokémons com mais de 100 de velocidade
pkmn.filter(
    pkmn["Speed"] > 100
).select(
    "Type 1"
).distinct().display()

Type 1
Grass
Fire
Water
Bug
Normal
Poison
Electric
Ground
Fighting
Psychic


In [0]:
# Vamos alterar os casos onde Speed é superior a 100 para Fire
pkmn = pkmn.withColumn(
    "Type 1",
    F.when(
        pkmn["Speed"] > 100, "Fire"
    ).otherwise(
        pkmn["Type 1"]
    )
)

In [0]:
# Observe como os valores mudaram
pkmn.filter(
    pkmn["Speed"] > 100
).select(
    "Type 1"
).distinct().display()

Type 1
Fire


Relendo o arquivo para desconsiderar os tratamentos de exemplos que fizemos acima

In [0]:

pkmn = spark.table("workspace.bronze.pokemon_data")

# Renomeando as colunas
pkmn = (
    pkmn
    .withColumnRenamed("Type 1", "Type_1")
    .withColumnRenamed("Type 2", "Type_2")
    .withColumnRenamed("Sp. Atk", "Sp_Atk")
    .withColumnRenamed("Sp. Def", "Sp_Def")
)



## 3. Operações em grupo

Com PySpark nós podemos aplicar operações em grupos usando o método *.groupby()*. Ele é muito útil por ser uma forma bem simples de extrair informação de dados agregados. Para utilizá-lo, passamos as colunas nas quais queremos agrupar os dados e a operação que queremos fazer. Para exemplificar, vamos ver quantos Pokémons lendários cada geração tem:

In [0]:
pkmn_soma = (pkmn
            .groupBy("Generation") # Campo que sera agrupado
            .agg(
                F.sum(F.col("Legendary").cast("int")) # Converte a coluna "Legendary" em inteiro e faz a soma
                .alias("Qtd_Legendary") # Nomeando a coluna que receberá o resultado da soma
                )
            )
pkmn_soma.display()

Generation,Qtd_Legendary
1,6
2,5
3,18
4,13
5,15
6,8


Podemos obter um relatório da média de diversas colunas para cada tipo de Pokémon:

In [0]:
pkmn_media = (pkmn
                .groupBy("Type_1")
                .agg(
                    F.mean("HP").alias("HP_medio"),
                    F.mean("Attack").alias("Attack_medio"),
                    F.mean("Defense").alias("Defense_medio")
                    )
                )
pkmn_media.display()

Type_1,HP_medio,Attack_medio,Defense_medio
Grass,67.27142857142857,73.21428571428571,70.8
Fire,69.90384615384616,84.76923076923077,67.76923076923077
Water,72.0625,74.15178571428571,72.94642857142857
Bug,56.88405797101449,70.97101449275362,70.72463768115942
Normal,77.27551020408163,73.46938775510205,59.8469387755102
Poison,67.25,74.67857142857143,68.82142857142857
Electric,59.79545454545455,69.0909090909091,66.29545454545455
Ground,73.78125,95.75,84.84375
Fairy,74.11764705882354,61.529411764705884,65.70588235294117
Fighting,69.85185185185185,96.77777777777777,65.92592592592592




###  Exercício 2
Use o método *.groupby()* para descobrir qual país tem o melhor *overall* médio. Crie a coluna 'avg_overall'

Seu df country_avg_overall deve conter as seguintes colunas:
- `nationality`
- `overall`
- `avg_overall`

In [0]:
fut_players = spark.table("workspace.bronze.fut_players_data")

country_avg_overall = (
    fut_players
    .groupBy("nationality")
    .agg(
        (F.avg(F.col("overall"))).alias("avg_overall")
    )
)

# Retornar a nacionalidade com maior overall médio e o overall médio do brasil
melhor = (
    country_avg_overall
    .orderBy(F.col("avg_overall").desc())
    .limit(1)
    .collect()[0]
)

brasil = (
    country_avg_overall
    .filter(F.col("nationality") == "Brazil")
    .collect()[0]
)

display({
    "Melhor overall médio": f"{melhor['nationality']}: {melhor['avg_overall']:.2f}",
    "Overall médio do Brasil": round(brasil['avg_overall'], 2)
})

{'Melhor overall médio': 'Dominican Republic: 79.00',
 'Overall médio do Brasil': 75.22}

Agora nós já cobrimos toda a parte básica do Spark! Vamos praticar essa última parte!

### Exercício 2.1
Crie um racional que retorne a classificação para o jogador de acordo com as instruções abaixo, então aplique isso para o dataframe fut_players.

*Observação:* considere os limites dentro do intervalo de classificação.
exemplo

-50 contém todos os valores menores que 50 e o valor 50 incluso;


51-60 contém todos os valores entre 51 e 60 com os limites [51,60] inclusos no grupo;


e assim por diante ...

In [0]:

"""
    Através do overall do jogador retorne a classificação conforme a seguir:
    Overall -> classification
    -50     -> "Amador"
    51-60   -> "Ruim"
    61-70   -> "Ok"
    71-80   -> "Bom"
    81-90   -> "Ótimo"
    91+     -> "Lenda"
    
    I: int overall
    O: string
"""

fut_players = spark.table("workspace.bronze.fut_players_data")

fut_players_classification = fut_players.withColumn(
    "classification",
    F.when(
        F.col("overall") <= 50,
        "Amador"
    ).when(
        (F.col("overall") <= 60),
        "Ruim"
    ).when(
        (F.col("overall") <= 70),
        "Ok"
    ).when(
        (F.col("overall") <= 80),
        "Bom"
    ).when(
        (F.col("overall") <= 90),
        "Ótimo"
    ).otherwise(
        "Lenda"
    )
)

# Contar quantos jogadores há em cada classificação
fut_players_classification.groupBy("classification").count().orderBy("count", ascending=False).display()

classification,count
Ok,9500
Bom,4785
Ruim,2788
Ótimo,1482
Lenda,141
Amador,135


## Desafio — Montando o Time dos Sonhos do 🇧🇷

Ainda utilizando a base **`fut_players_data`**, imagine que você é um grande fã do jogo *FIFA*, e deseja montar o **Time dos Sonhos (Dream Team)** do **Brasil**, selecionando os **melhores jogadores por posição**, ou seja, aqueles com o **maior overall** dentro de cada grupo de posição.

Para isso, adote a **formação tática 4-4-2**, composta por:

- **1 Goleiro (GK)**  
- **4 Defensores (Defesa)**  
- **4 Meio-campistas (Meio)**  
- **2 Atacantes (Ataque)**  

### Objetivo
Criar um *DataFrame* com **11 linhas**, representando o **melhor jogador de cada posição dentro da formação 4-4-2**, com as seguintes colunas:

- `nationality` — nacionalidade do jogador  
- `position_group` — posição agrupada (Goleiro, Defesa, Meio, Ataque)  
- `player_name` — nome do jogador  
- `overall` — nota geral (overall)

---

### Agrupamento de posições
Para facilitar a análise, agrupe as posições originais da base conforme a tabela abaixo:

| **position_group** | **Posições incluídas (`position`)** | **Descrição** |
|:--------------------|:------------------------------------|:---------------|
| **Goleiro** | `GK` | Jogadores que atuam exclusivamente no gol. |
| **Defesa** | `CB`, `LB`, `RB`, `LWB`, `RWB` | Zagueiros e laterais (defensores). |
| **Meio** | `CM`, `CDM`, `CAM`, `LM`, `RM` | Meio-campistas centrais, volantes e meias ofensivos/laterais. |
| **Ataque** | `ST`, `CF`, `LW`, `RW`, `LF`, `RF` | Atacantes e pontas. |
| **Outros** | *(demais posições não classificadas)* | Jogadores fora do esquema tático principal (ex: cartas especiais). |

---

### 🏁 Entrega esperada
Seu *DataFrame final* deve retornar **11 jogadores**, representando o **Time dos Sonhos do Brasil (formação 4-4-2)**, conforme os critérios acima.

In [0]:
fut_players = spark.table("workspace.bronze.fut_players_data")

In [0]:
gk = (
    fut_players.filter(F.col("position") == "GK")
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

gk = gk.withColumn("position", F.lit("Goleiro"))

display(gk.limit(5))

nationality,position,player_name,overall
Spain,Goleiro,De Gea,97
Russia,Goleiro,Yashin,94
Spain,Goleiro,De Gea,93
Denmark,Goleiro,Schmeichel,92
Spain,Goleiro,De Gea,92


In [0]:
defense = (
    fut_players.filter(F.col("position").isin(["CB", "RB", "LB", "RWB", "LWB"]))
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

defense = defense.withColumn("position", F.lit("Defesa"))

display(defense.limit(5))

nationality,position,player_name,overall
Spain,Defesa,Ramos,97
France,Defesa,Varane,96
Holland,Defesa,Van Dijk,95
Italy,Defesa,Maldini,94
Brazil,Defesa,Marcelo,94


In [0]:
midfield = (
    fut_players.filter(F.col("position").isin(["CDM", "CM", "RM", "LM", "CAM"]))
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

midfield = midfield.withColumn("position", F.lit("Meio"))

display(midfield.limit(5))

nationality,position,player_name,overall
Croatia,Meio,Modric,99
Brazil,Meio,Pelé,98
Argentina,Meio,Maradona,97
Belgium,Meio,De Bruyne,97
France,Meio,Kanté,96


In [0]:
attack = (
    fut_players.filter(F.col("position").isin(["ST", "CF", "LW", "RW", "RF", "LF"]))
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

attack = attack.withColumn("position", F.lit("Ataque"))

display(attack.limit(5))

nationality,position,player_name,overall
Argentina,Ataque,Messi,99
Portugal,Ataque,Ronaldo,99
Brazil,Ataque,Neymar Jr,97
France,Ataque,Mbappé,97
Brazil,Ataque,Ronaldo,96


In [0]:
team = (
    gk.orderBy(F.desc("overall")).limit(1)
    .unionByName(defense.orderBy(F.desc("overall")).limit(4))
    .unionByName(midfield.orderBy(F.desc("overall")).limit(4))
    .unionByName(attack.orderBy(F.desc("overall")).limit(2))
)

display(team)

nationality,position,player_name,overall
Spain,Goleiro,De Gea,97
Spain,Defesa,Ramos,97
France,Defesa,Varane,96
Holland,Defesa,Van Dijk,95
Italy,Defesa,Maldini,94
Croatia,Meio,Modric,99
Brazil,Meio,Pelé,98
Argentina,Meio,Maradona,97
Belgium,Meio,De Bruyne,97
Portugal,Ataque,Ronaldo,99


### Desafio Bônus

Você deve ter notado que **Neymar** aparece tanto entre os melhores jogadores de **ataque** quanto do **meio-campo**.  
Isso acontece porque o dataset contém **múltiplas versões do mesmo jogador**, inclusive atuando em **outras posições**, o que é típico dos modos do *FIFA/Ultimate Team*.

O seu desafio agora é **refazer o exercício anterior**, garantindo que **cada jogador apareça apenas uma vez** no *DataFrame final*.

- Caso o jogador possua mais de uma versão (carta), **considere apenas aquela com o maior valor de `overall`**.  
- Em seguida, **reaplique a lógica da formação 4-4-2**, selecionando os melhores por grupo de posição.

---

### 🏁 Entrega Esperada
Seu *DataFrame final* deve retornar **11 jogadores únicos**, representando o **Dream Team do Brasil** na **formação tática 4-4-2**, **sem repetição de atletas**, conforme os critérios estabelecidos acima.


In [0]:
# checando duplicatas de neymar

ney = (
    fut_players
    .filter(F.col("player_name").startswith("Ney"))
)

display(ney)

player_id,player_name,player_extended_name,quality,revision,origin,overall,club,league,nationality,position,age,date_of_birth,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,gk_diving,gk_reflexes,gk_handling,gk_speed,gk_kicking,gk_positoning,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,cb,rb,lb,rwb,lwb,cdm,cm,rm,lm,cam,cf,rf,lf,rw,lw,st,traits,specialities,base_id
846,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,Normal,null,92,Paris Saint-Germain,Ligue 1 Conforama,Brazil,LW,27,1992-02-05,175,68,5,2018-09-19,92.0,94,90,95.0,96,84,94,95,96,94,84.0,89,87,80,82,84,81,83.0,87,79,87,84,78,88,32.0,36,62,27,24,33,59.0,61,81,49,56,null,null,null,null,null,null,Right,High,Med,5,5,47.0,61.0,61.0,66.0,66.0,61.0,81.0,89.0,89.0,90.0,90.0,90.0,90.0,90.0,90.0,84.0,"Flair, Takes Finesse Free Kicks","Speedster, Dribbler, FK Specialist, Acrobat, Clinical Finisher, Complete Forward",190871
16951,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,CL,null,93,Paris Saint-Germain,Ligue 1 Conforama,Brazil,LW,27,1992-02-05,175,68,5,2018-09-28,93.0,95,90,96.0,97,84,94,96,97,94,85.0,90,88,80,82,85,81,84.0,88,79,88,85,78,89,33.0,37,63,27,24,34,60.0,62,82,49,56,null,null,null,null,null,null,Right,High,Med,5,5,48.0,61.0,61.0,66.0,66.0,61.0,82.0,90.0,90.0,90.0,90.0,90.0,90.0,91.0,91.0,85.0,"Flair, Speed Dribbler (CPU AI Only), Technical Dribbler (CPU AI Only)","Speedster, Dribbler, FK Specialist, Acrobat, Clinical Finisher, Complete Forward",190871
17014,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,IF,TOTW3,93,Paris Saint-Germain,Ligue 1 Conforama,Brazil,LF,27,1992-02-05,175,68,5,2018-10-03,93.0,95,90,96.0,97,84,94,96,97,94,86.0,91,89,81,83,86,82,85.0,89,80,89,86,79,90,33.0,37,63,27,24,34,60.0,62,82,49,56,null,null,null,null,null,null,Right,High,Med,5,5,48.0,62.0,62.0,67.0,67.0,62.0,83.0,90.0,90.0,91.0,91.0,91.0,91.0,91.0,91.0,85.0,"Flair, Speed Dribbler (CPU AI Only), Technical Dribbler (CPU AI Only)","Speedster, Dribbler, FK Specialist, Acrobat, Clinical Finisher, Complete Forward",190871
17388,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,TOTGS,TOTGS,94,Paris Saint-Germain,Ligue 1 Conforama,Brazil,CAM,27,1992-02-05,175,68,5,2018-12-07,94.0,96,91,97.0,98,85,95,97,98,95,88.0,93,91,83,85,88,84,87.0,91,82,91,88,81,92,34.0,38,65,28,25,35,62.0,64,85,51,58,null,null,null,null,null,null,Right,High,Med,5,5,49.0,63.0,63.0,68.0,68.0,63.0,84.0,92.0,92.0,93.0,92.0,92.0,92.0,93.0,93.0,87.0,"Flair, Speed Dribbler (CPU AI Only), Technical Dribbler (CPU AI Only)","Speedster, Dribbler, Play Maker, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward",190871
17616,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,TOTY Nominee Loan,null,92,Paris Saint-Germain,Ligue 1 Conforama,Brazil,LW,27,1992-02-05,175,68,5,2019-01-10,92.0,94,90,95.0,96,84,94,95,96,94,84.0,89,87,80,82,84,81,83.0,87,79,87,84,78,88,32.0,36,62,27,24,33,59.0,61,81,49,56,null,null,null,null,null,null,Right,High,Med,5,5,47.0,61.0,61.0,66.0,66.0,61.0,81.0,89.0,89.0,90.0,90.0,90.0,90.0,90.0,90.0,84.0,"Flair, Speed Dribbler (CPU AI Only), Technical Dribbler (CPU AI Only)","Speedster, Dribbler, FK Specialist, Acrobat, Clinical Finisher, Complete Forward",190871
17620,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,TOTY,TOTY,97,Paris Saint-Germain,Ligue 1 Conforama,Brazil,LW,27,1992-02-05,175,68,5,2019-01-11,97.0,99,94,99.0,99,87,97,99,99,97,95.0,99,98,90,92,95,91,94.0,98,89,98,95,88,99,45.0,50,87,37,33,46,80.0,82,99,66,75,null,null,null,null,null,null,Right,High,Med,5,5,60.0,71.0,71.0,76.0,76.0,72.0,91.0,96.0,96.0,97.

In [0]:
fut_without_duplicates = (
    fut_players
    .orderBy(F.col("overall").desc()) # garante o maior over dos jogadores
    .dropDuplicates(["player_name"])
)

display(fut_without_duplicates.limit(5))

player_id,player_name,player_extended_name,quality,revision,origin,overall,club,league,nationality,position,age,date_of_birth,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,gk_diving,gk_reflexes,gk_handling,gk_speed,gk_kicking,gk_positoning,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,cb,rb,lb,rwb,lwb,cdm,cm,rm,lm,cam,cf,rf,lf,rw,lw,st,traits,specialities,base_id
17575,Ronaldo,C. Ronaldo dos Santos Aveiro,Gold - Rare,TOTY,TOTY,99,Juventus,Serie A TIM,Portugal,ST,34,1985-02-05,187,83,5,2019-01-07,97.0,95,98,98.0,94,76,99,99,95,99,99.0,99,99,99,99,92,90,95.0,96,98,89,95,90,95,50.0,41,99,40,44,32,95.0,99,99,95,75,null,null,null,null,null,null,Right,High,Low,4,5,65.0,71.0,71.0,75.0,75.0,73.0,91.0,97.0,97.0,97.0,98.0,98.0,98.0,97.0,97.0,98.0,"Power Free-Kick, Flair, Long Shot Taker (CPU AI Only), Speed Dribbler (CPU AI Only)","Speedster, Aerial Threat, Dribbler, Distance Shooter, Crosser, FK Specialist, Acrobat, Strength, Clinical Finisher, Complete Forward, Poacher",20801
17576,Messi,Lionel Messi,Gold - Rare,TOTY,TOTY,99,FC Barcelona,LaLiga Santander,Argentina,CF,31,1987-06-24,170,72,5,2019-01-07,97.0,99,94,99.0,93,97,97,99,99,99,99.0,99,99,92,99,93,81,98.0,99,85,99,99,96,99,45.0,30,98,46,39,36,85.0,94,99,82,66,null,null,null,null,null,null,Left,Med,Med,4,4,62.0,69.0,69.0,73.0,73.0,71.0,92.0,97.0,97.0,98.0,98.0,98.0,98.0,97.0,97.0,97.0,"Finesse Shot, Long Shot Taker (CPU AI Only), Speed Dribbler (CPU AI Only), Playmaker (CPU AI Only), One Club Player, Chip Shot (CPU AI Only)","Speedster, Aerial Threat, Dribbler, Distance Shooter, FK Specialist, Acrobat, Clinical Finisher, Complete Forward, Poacher",158023
17580,Modric,Luka Modric,Gold - Rare,TOTY,TOTY,99,Real Madrid,LaLiga Santander,Croatia,CM,33,1985-09-09,172,66,4,2019-01-08,90.0,94,85,99.0,99,99,97,99,97,91,94.0,97,89,97,99,94,99,99.0,99,94,85,99,96,93,94.0,99,73,80,99,98,90.0,91,99,77,83,null,null,null,null,null,null,Right,High,High,4,4,90.0,94.0,94.0,96.0,96.0,95.0,98.0,96.0,96.0,97.0,96.0,96.0,96.0,96.0,96.0,92.0,"Leadership, Long Passer (CPU AI Only), Long Shot Taker (CPU AI Only), Playmaker (CPU AI Only), Technical Dribbler (CPU AI Only)","Dribbler, Play Maker, Distance Shooter, Crosser, Tackler, Tactician, Acrobat, Clinical Finisher, Complete Defender, Midfielder",177003
1,Pelé,Arantes Nascimento Edson,Gold - Rare,Icon,Prime,98,Icons,Icons,Brazil,CAM,78,1940-10-23,173,70,0,2018-09-19,95.0,95,95,96.0,94,93,98,97,96,98,96.0,97,98,94,94,95,93,93.0,97,90,89,96,88,89,60.0,67,94,55,53,49,76.0,88,86,76,59,null,null,null,null,null,null,Right,High,Med,4,5,70.0,77.0,77.0,79.0,79.0,77.0,91.0,95.0,95.0,96.0,96.0,96.0,96.0,96.0,96.0,95.0,Finesse Shot,"Speedster, Aerial Threat, Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward, Poacher",237067
2,Maradona,Diego Maradona,Gold - Rare,Icon,Prime,97,Icons,Icons,Argentina,CAM,58,1960-10-30,165,70,0,2018-09-19,92.0,94,90,97.0,91,98,94,97,98,95,93.0,92,97,85,94,88,94,92.0,95,88,96,93,89,96,40.0,44,67,27,42,37,76.0,82,78,75,76,null,null,null,null,null,null,Left,High,Med,3,5,57.0,66.0,66.0,71.0,71.0,69.0,88.0,93.0,93.0,95.0,94.0,94.0,94.0,94.0,94.0,90.0,"Avoids Using Weaker Foot, Finesse Shot, Flair, Takes Finesse Free Kicks","Speedster, Dribbler, Play Maker, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Midfielder, Complete Forward",190042


In [0]:
# checando duplicatas de neymar pós tratamento de duplicatas

ney = (
    fut_without_duplicates
    .filter(F.col("player_name").startswith("Ney"))
)

display(ney)

player_id,player_name,player_extended_name,quality,revision,origin,overall,club,league,nationality,position,age,date_of_birth,height,weight,intl_rep,added_date,pace,pace_acceleration,pace_sprint_speed,dribbling,drib_agility,drib_balance,drib_reactions,drib_ball_control,drib_dribbling,drib_composure,shooting,shoot_positioning,shoot_finishing,shoot_shot_power,shoot_long_shots,shoot_volleys,shoot_penalties,passing,pass_vision,pass_crossing,pass_free_kick,pass_short,pass_long,pass_curve,defending,def_interceptions,def_heading,def_marking,def_stand_tackle,def_slid_tackle,physicality,phys_jumping,phys_stamina,phys_strength,phys_aggression,gk_diving,gk_reflexes,gk_handling,gk_speed,gk_kicking,gk_positoning,pref_foot,att_workrate,def_workrate,weak_foot,skill_moves,cb,rb,lb,rwb,lwb,cdm,cm,rm,lm,cam,cf,rf,lf,rw,lw,st,traits,specialities,base_id
17620,Neymar Jr,Neymar da Silva Santos Jr.,Gold - Rare,TOTY,TOTY,97,Paris Saint-Germain,Ligue 1 Conforama,Brazil,LW,27,1992-02-05,175,68,5,2019-01-11,97.0,99,94,99.0,99,87,97,99,99,97,95.0,99,98,90,92,95,91,94.0,98,89,98,95,88,99,45.0,50,87,37,33,46,80.0,82,99,66,75,null,null,null,null,null,null,Right,High,Med,5,5,60.0,71.0,71.0,76.0,76.0,72.0,91.0,96.0,96.0,97.0,97.0,97.0,97.0,97.0,97.0,94.0,"Flair, Speed Dribbler (CPU AI Only), Technical Dribbler (CPU AI Only)","Speedster, Dribbler, Distance Shooter, Crosser, FK Specialist, Acrobat, Clinical Finisher, Complete Forward, Poacher",190871


In [0]:
gk = (
    fut_without_duplicates.filter(F.col("position") == "GK")
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

gk = gk.withColumn("position", F.lit("Goleiro"))

defense = (
    fut_without_duplicates.filter(F.col("position").isin(["CB", "RB", "LB", "RWB", "LWB"]))
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

defense = defense.withColumn("position", F.lit("Defesa"))

midfield = (
    fut_without_duplicates.filter(F.col("position").isin(["CDM", "CM", "RM", "LM", "CAM"]))
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

midfield = midfield.withColumn("position", F.lit("Meio"))

attack = (
    fut_without_duplicates.filter(F.col("position").isin(["ST", "CF", "LW", "RW", "RF", "LF"]))
    .select("nationality", "position", "player_name", "overall")
    .orderBy(F.col("overall").desc())
)

attack = attack.withColumn("position", F.lit("Ataque"))

team = (
    gk.orderBy(F.desc("overall")).limit(1)
    .unionByName(defense.orderBy(F.desc("overall")).limit(4))
    .unionByName(midfield.orderBy(F.desc("overall")).limit(4))
    .unionByName(attack.orderBy(F.desc("overall")).limit(2))
)

display(team)

nationality,position,player_name,overall
Spain,Goleiro,De Gea,97
Spain,Defesa,Ramos,97
France,Defesa,Varane,96
Holland,Defesa,Van Dijk,95
Brazil,Defesa,Marcelo,94
Croatia,Meio,Modric,99
Brazil,Meio,Pelé,98
Belgium,Meio,De Bruyne,97
Argentina,Meio,Maradona,97
Argentina,Ataque,Messi,99


# Declaração de Inexistência de Plágio:

1. Eu sei que plágio é utilizar o trabalho de outra pessoa e apresentar como meu.
2. Eu sei que plágio é errado e declaro que este notebook foi feito por mim.
3. Tenho consciência de que a utilização do trabalho de terceiros é antiético e está sujeito a medidas administrativas.
4. Declaro também que não compartilhei e não compartilharei meu trabalho com o intuito de que seja copiado e submetido por outra pessoa.

# Fim da aula!

Obrigado por participar do curso, você acaba de finalizar o Módulo de Pyspark. Neste momento você já deve ser capaz de manipular seus dados no Spark, utilizando as bibliotecas que acabamos de aprender!

Lembre-se que sempre que surgir alguma dúvida, você pode olhar a documentação do [PySpark](https://spark.apache.org/docs/latest/api/python/reference).